In [ ]:
## Experimental Setup

In [ ]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback 

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

In [ ]:
from gepcore.utils import cell_graph
from gepnet.model_v2 import get_gepnet, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [ ]:
from pygraphviz import AGraph
import glob

In [ ]:
graph = [AGraph(g) for g in glob.glob('comp_graphs/experiment_1/best/indv_1/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
import random
seed = 200
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# enable torch backends
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=40,
                   repeat_list=[2, 3, 1],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
path = untar_data(URLs.CIFAR)
tfms = get_transforms() #xtra_tfms=[cutout()]
bs = 256
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
model_dir = '/home/cliff/ResearchProjects/models/evolution_search/experiment_1/'
learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=model_dir).load(model_dir+'experiment_1_model_1b').mixup() 
learn.to_fp16()   

In [ ]:
learn.lr_find(wd=4e-4,end_lr=100)
learn.recorder.plot()

In [ ]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='experiment_1_model_1c')
learn.fit_one_cycle(800, 3e-3, wd=0.0004, callbacks=[cb]) 

In [ ]:
####################### Testing ###########################

In [ ]:
graph_dir = '/content/mnt/My Drive/Colab Notebooks/ResearchProject/comp_graphs/experiment_1/best/indv_1/*.dot'
graph = [AGraph(g) for g in glob.glob(graph_dir)]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=40,
                   repeat_list=[2, 3, 1],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
tfms = get_transforms(do_flip=False)
path = untar_data(URLs.CIFAR)
bs = 256

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
model_dir = '/content/mnt/My Drive/Colab Notebooks/ResearchProject/models/'
model = Learner(data, net, metrics=[accuracy, error_rate]).load(model_dir+'experiment_1_model_1b')
#model

In [ ]:
model.validate()

In [ ]:
preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
preds[:5]